In [1]:
%pip install -q -U google-generativeai
%pip install youtube_transcript_api
%pip install PyTube openai

import json
from google.colab import userdata

import os
import shutil
import subprocess

import google.generativeai as genai
from openai import OpenAI

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

OPENAI_KEY = userdata.get('OPENAI_KEY')
client = OpenAI(
    api_key=OPENAI_KEY
)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube
import moviepy.editor as mp
import re

In [ ]:
base_prompt = "Extract a list of all {topic_prompt} and the corresponding start and end timestamps where they are discussed in the following YouTube transcript. Ensure duration is within 20 to 80 seconds with recommended length of 30 sec,it is IMPORTANT video length should not be less than 20 sec and should be bound by limits. Format the output as a JSON array:"

output_format = """

[
    {
      "title": "string",
      "info": "string",
      "why_clip_was_chosen": "string",
      "url":string,
      "start_time": int,
      "end_time": int,
    },
    ...
]


"""

additional_base_prompt = """
if nothing can be extracted return an empty array i.e. []

{url_source_prompt}

dont return anything else as output it should always be an array

Transcript:

"""


def get_message(transcript, topic_prompt="Books", url_source=""):
  url_source_prompt = ""
  if url_source:
    url_source_prompt = f"\nFor the URL, use the {url_source} link relevant to the discussed topic."
  else:
    url_source_prompt = "keep the 'url' filed empty don't put any value in that key"


  base_prompt_with_topic = base_prompt.format(topic_prompt=topic_prompt)
  additional_base_prompt_with_url_source_prompt = additional_base_prompt.format(url_source_prompt=url_source_prompt)

  transcript_text = "\n".join([f"{item['start']} - {item['text']}" for item in transcript])
  result = base_prompt_with_topic + output_format + additional_base_prompt_with_url_source_prompt + transcript_text
  return result

In [ ]:
def get_video_id(url):
    # Extract video id from YouTube URL
    video_id = re.search(r'(?<=v=)[^&#]+', url)
    if video_id is None:
        video_id = re.search(r'(?<=be/)[^&#]+', url)
    if video_id:
        return video_id.group(0)
    else:
        raise ValueError("Invalid YouTube URL")

def get_raw_transcript(url):
    video_id = get_video_id(url)
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    return transcript_list

def get_llm_output(messageText, service = "gemini", model = ""):
  if service == "gemini":
      model = genai.GenerativeModel(model)
      output = model.generate_content(messageText)
      return output.text
  elif service == "openai":
      chat_completion = client.chat.completions.create(
        messages=[
          {
            "role": "user",
            "content": messageText,
          }
        ],
        model=model,
      )
      return chat_completion.choices[0].message.content
  else:
    return "[]"

In [ ]:
def get_clips(url, topic_prompt, service ="gemini", model="gemini-pro", url_source = ""):
  transcript = get_raw_transcript(url)
  message = get_message(transcript, topic_prompt, url_source)
  llmOutput = get_llm_output(message ,service,model)
  llmOutput = llmOutput.replace("```", "")
  llmOutput = llmOutput.replace("json", "")
  result = json.loads(llmOutput)
  return result


In [ ]:
url = "https://www.youtube.com/watch?v=HMX0vrJj5nM"
extracted_clips = get_clips(
    url = url,
    topic_prompt = "gadgets mentioned",
    service = "gemini",
    model = "gemini-pro",
    url_source = ""
  )

In [ ]:
def download_and_trim(url, base_directory, content_list):
    try:

        # If the directory already exists, remove it first
        if os.path.exists(base_directory):
            shutil.rmtree(base_directory)

        yt = YouTube(url)
        video = yt.streams.first().download(output_path=base_directory)
        print(f"Downloaded video: {video}")


        # Ensure the clips directory exists
        clips_directory = os.path.join(base_directory, "clips")
        os.makedirs(clips_directory, exist_ok=True)

        for content in content_list:
            start_time = content["start_time"]
            end_time = content["end_time"]
            title = content["title"]
            output_file = os.path.join(clips_directory, f"{title}.mp4")

            # Construct the ffmpeg command
            ffmpeg_command = [
                "ffmpeg",
                "-i", video,
                "-ss", str(start_time),
                "-to", str(end_time),
                "-c", "copy",
                output_file
            ]

            # Run the ffmpeg command
            subprocess.run(ffmpeg_command, check=True)

    except Exception as e:
        print(f"An error occurred: {e}")

base_directory = "/content/downloads/io"

download_and_trim(
    url = url,
    base_directory = base_directory,
    content_list = extracted_clips
  )

Downloaded video: /content/downloads/io/Ive tested 5000 gadgets - These are the 10 I actually Use.mp4


# *Editing starts*

In [ ]:
%pip install ffmpeg-python==0.2.0

In [ ]:
videofilename = "/content/16-inch MacBook with M1 Max Chip.mp4"

In [ ]:
import ffmpeg

audiofilename = videofilename.replace(".mp4",'.mp3')

# Create the ffmpeg input stream
input_stream = ffmpeg.input(videofilename)

# Extract the audio stream from the input stream
audio = input_stream.audio

# Save the audio stream as an MP3 file
output_stream = ffmpeg.output(audio, audiofilename)

# Overwrite output file if it already exists
output_stream = ffmpeg.overwrite_output(output_stream)

ffmpeg.run(output_stream)

print (audiofilename)


/content/16-inch MacBook with M1 Max Chip.mp3


In [ ]:
from IPython.display import Audio

Audio(audiofilename)


In [ ]:
%pip install faster-whisper==0.7.0

In [ ]:
from faster_whisper import WhisperModel

model_size = "medium"
model = WhisperModel(model_size)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
segments, info = model.transcribe(audiofilename, word_timestamps=True)
segments = list(segments)  # The transcription will actually run here.
for segment in segments:
    for word in segment.words:
        print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))

[0.86s -> 1.30s]  This
[1.30s -> 1.74s]  is
[1.74s -> 1.82s]  the
[1.82s -> 2.06s]  best
[2.06s -> 2.34s]  laptop
[2.34s -> 2.54s]  I've
[2.54s -> 2.80s]  ever
[2.80s -> 3.06s]  used
[3.06s -> 3.22s]  for
[3.22s -> 3.46s]  two
[3.46s -> 3.90s]  reasons.
[4.12s -> 4.44s]  One,
[4.64s -> 4.80s]  that
[4.80s -> 4.94s]  because
[4.94s -> 5.16s]  this
[5.16s -> 5.30s]  is
[5.30s -> 5.46s]  using
[5.46s -> 5.68s]  an
[5.68s -> 5.90s]  Apple
[5.90s -> 6.14s] -made
[6.14s -> 6.60s]  chip,
[6.86s -> 6.88s]  which
[6.88s -> 7.06s]  is
[7.06s -> 7.18s]  based
[7.18s -> 7.32s]  on
[7.32s -> 7.46s]  the
[7.46s -> 7.62s]  more
[7.62s -> 8.00s]  efficient
[8.00s -> 8.48s]  mobile
[8.48s -> 8.86s]  phone
[8.86s -> 9.26s]  processor
[9.26s -> 9.82s]  architecture,
[10.10s -> 10.22s]  the
[10.22s -> 10.38s]  battery
[10.38s -> 10.56s]  life
[10.56s -> 10.62s]  in
[10.62s -> 10.86s]  this
[10.86s -> 11.18s]  literally
[11.18s -> 11.42s]  lasts
[11.42s -> 11.74s]  twice
[11.74s -> 11.96s]  as
[11.96s -> 1

In [ ]:
wordlevel_info = []

for segment in segments:
    for word in segment.words:
      wordlevel_info.append({'word':word.word,'start':word.start,'end':word.end})

wordlevel_info

[{'word': ' This', 'start': 0.860000000000001, 'end': 1.3000000000000005},
 {'word': ' is', 'start': 1.3000000000000005, 'end': 1.74},
 {'word': ' the', 'start': 1.74, 'end': 1.82},
 {'word': ' best', 'start': 1.82, 'end': 2.06},
 {'word': ' laptop', 'start': 2.06, 'end': 2.34},
 {'word': " I've", 'start': 2.34, 'end': 2.54},
 {'word': ' ever', 'start': 2.54, 'end': 2.8},
 {'word': ' used', 'start': 2.8, 'end': 3.06},
 {'word': ' for', 'start': 3.06, 'end': 3.22},
 {'word': ' two', 'start': 3.22, 'end': 3.46},
 {'word': ' reasons.', 'start': 3.46, 'end': 3.9},
 {'word': ' One,', 'start': 4.12, 'end': 4.44},
 {'word': ' that', 'start': 4.64, 'end': 4.8},
 {'word': ' because', 'start': 4.8, 'end': 4.94},
 {'word': ' this', 'start': 4.94, 'end': 5.16},
 {'word': ' is', 'start': 5.16, 'end': 5.3},
 {'word': ' using', 'start': 5.3, 'end': 5.46},
 {'word': ' an', 'start': 5.46, 'end': 5.68},
 {'word': ' Apple', 'start': 5.68, 'end': 5.9},
 {'word': '-made', 'start': 5.9, 'end': 6.14},
 {'wor

In [ ]:
from IPython.display import YouTubeVideo
from ipywidgets import widgets
from IPython.display import display

In [ ]:
# Create a table to display the wordlevel_info
table_columns = ['Start', 'End', 'Word']
# Create the table rows
table_rows = []

for word_info in wordlevel_info:
    start_widget = widgets.Text(value=str(word_info['start']))
    end_widget = widgets.Text(value=str(word_info['end']))
    word_widget = widgets.Text(value=word_info['word'])
    row_widgets = widgets.HBox([start_widget, end_widget, word_widget])
    table_rows.append(row_widgets)


# Create the update button
button = widgets.Button(description='Update')

# Create the output area for displaying the updated wordlevel_info
output_area = widgets.Output()


In [ ]:
# Create a new variable to store the modified wordlevel_info
modified_wordlevel_info = []

def update_wordlevel_info(event):
    with output_area:
        output_area.clear_output()
        updated_wordlevel_info = []

        for row_widgets in table_rows:
            start = float(row_widgets.children[0].value)  # Convert start to float
            end = float(row_widgets.children[1].value)  # Convert end to float
            word = row_widgets.children[2].value
            word = word.strip()
            updated_wordlevel_info.append({'start': start, 'end': end, 'word': word})

        # Print the updated wordlevel_info
        print(updated_wordlevel_info)

        # Assign updated_wordlevel_info to modified_wordlevel_info
        global modified_wordlevel_info
        modified_wordlevel_info = updated_wordlevel_info


button.on_click(update_wordlevel_info)
update_wordlevel_info({})
# Create the table widget
table_widget = widgets.VBox(table_rows)

# Display the UI elements
display(table_widget, button, output_area)


Button(description='Update', style=ButtonStyle())

Output()

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(modified_wordlevel_info, f,indent=4)

with open('data.json', 'r') as f:
    wordlevel_info_modified = json.load(f)


In [ ]:
def split_text_into_lines(data, MaxChars =30,MaxDuration=2.5, MaxGap= 1.5):

    # MaxChars
    # MaxDuration in seconds
    # MaxGap Split if nothing is spoken (gap) for these many seconds


    subtitles = []
    line = []
    line_duration = 0
    line_chars = 0


    for idx,word_data in enumerate(data):
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item["word"] for item in line)


        # Check if adding a new word exceeds the maximum character count or duration
        new_line_chars = len(temp)

        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars
        if idx>0:
          gap = word_data['start'] - data[idx-1]['end']
          # print (word,start,end,gap)
          maxgap_exceeded = gap > MaxGap
        else:
          maxgap_exceeded = False


        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item["word"] for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0
                line_chars = 0


    if line:
        subtitle_line = {
            "word": " ".join(item["word"] for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

In [ ]:
linelevel_subtitles = split_text_into_lines(wordlevel_info_modified)
for line in linelevel_subtitles:
  json_str = json.dumps(line, indent=4)
  print(json_str)

{
    "word": "This is the best laptop I've ever",
    "start": 0.860000000000001,
    "end": 2.8,
    "textcontents": [
        {
            "start": 0.860000000000001,
            "end": 1.3000000000000005,
            "word": "This"
        },
        {
            "start": 1.3000000000000005,
            "end": 1.74,
            "word": "is"
        },
        {
            "start": 1.74,
            "end": 1.82,
            "word": "the"
        },
        {
            "start": 1.82,
            "end": 2.06,
            "word": "best"
        },
        {
            "start": 2.06,
            "end": 2.34,
            "word": "laptop"
        },
        {
            "start": 2.34,
            "end": 2.54,
            "word": "I've"
        },
        {
            "start": 2.54,
            "end": 2.8,
            "word": "ever"
        }
    ]
}
{
    "word": "used for two reasons. One, that",
    "start": 2.8,
    "end": 4.8,
    "textcontents": [
        {
            "start

In [ ]:
%pip install moviepy==2.0.0.dev2 imageio==2.25.1

In [ ]:
!apt install imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml

In [ ]:
from moviepy.editor import TextClip, CompositeVideoClip, ColorClip
import numpy as np


def create_caption(textJSON, framesize,font = "Helvetica", color='white', highlight_color='pink', stroke_color='black',stroke_width=1.5):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end']-textJSON['start']

    word_clips = []
    xy_textclips_positions =[]

    x_pos = 0
    y_pos = 0
    line_width = 0  # Total width of words in the current line
    frame_width = framesize[0]
    frame_height = framesize[1]

    x_buffer = frame_width*1/10

    max_line_width = frame_width - 2 * (x_buffer)

    fontsize = int(frame_height * 0.075) #7.5 percent of video height

    space_width = ""
    space_height = ""

    for index,wordJSON in enumerate(textJSON['textcontents']):
      duration = wordJSON['end']-wordJSON['start']
      word_clip = TextClip(wordJSON['word'], font = font,fontsize=fontsize, color=color,stroke_color=stroke_color,stroke_width=stroke_width).set_start(textJSON['start']).set_duration(full_duration)
      word_clip_space = TextClip(" ", font = font,fontsize=fontsize, color=color).set_start(textJSON['start']).set_duration(full_duration)
      word_width, word_height = word_clip.size
      space_width,space_height = word_clip_space.size
      if line_width + word_width+ space_width <= max_line_width:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos,
                "y_pos": y_pos,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos, y_pos))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width, y_pos))

            x_pos = x_pos + word_width+ space_width
            line_width = line_width+ word_width + space_width
      else:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos+ word_height+10
            line_width = word_width + space_width

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos,
                "y_pos": y_pos,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.set_position((x_pos, y_pos))
            word_clip_space = word_clip_space.set_position((x_pos+ word_width , y_pos))
            x_pos = word_width + space_width


      word_clips.append(word_clip)
      word_clips.append(word_clip_space)


    for highlight_word in xy_textclips_positions:

      word_clip_highlight = TextClip(highlight_word['word'], font = font,fontsize=fontsize, color=highlight_color,stroke_color=stroke_color,stroke_width=stroke_width).set_start(highlight_word['start']).set_duration(highlight_word['duration'])
      word_clip_highlight = word_clip_highlight.set_position((highlight_word['x_pos'], highlight_word['y_pos']))
      word_clips.append(word_clip_highlight)

    return word_clips,xy_textclips_positions




In [ ]:
from moviepy.editor import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip
input_video = VideoFileClip(videofilename)
frame_size = input_video.size

all_linelevel_splits=[]

for line in linelevel_subtitles:
  out_clips,positions = create_caption(line,frame_size)

  max_width = 0
  max_height = 0

  for position in positions:
    # print (out_clip.pos)
    # break
    x_pos, y_pos = position['x_pos'],position['y_pos']
    width, height = position['width'],position['height']

    max_width = max(max_width, x_pos + width)
    max_height = max(max_height, y_pos + height)

  color_clip = ColorClip(size=(int(max_width*1.1), int(max_height*1.1)),
                       color=(64, 64, 64))
  color_clip = color_clip.set_opacity(.6)
  color_clip = color_clip.set_start(line['start']).set_duration(line['end']-line['start'])

  # centered_clips = [each.set_position('center') for each in out_clips]

  clip_to_overlay = CompositeVideoClip([color_clip]+ out_clips)
  clip_to_overlay = clip_to_overlay.set_position("bottom")


  all_linelevel_splits.append(clip_to_overlay)

input_video_duration = input_video.duration


final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

# Set the audio of the final video to be the same as the input video
final_video = final_video.set_audio(input_video.audio)

# Save the final clip as a video file with the audio included
final_video.write_videofile("/app/output.mp4", fps=24, codec="libx264", audio_codec="aac")

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open("output.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.